# Astropy Cosmologies

Astropy includes a convenient framework for defining cosmologies and working with realizations thereof.  These realizations hold the cosmological parameters and enable computation of a variety of redshift-parameterized properties, e.g. the luminosity distance.

For more information about the features presented below, please see the
[astropy.cosmology](https://docs.astropy.org/en/stable/cosmology/index.html) docs.

Also note that this tutorial assumes you have little or no knowledge of the astropy cosmology docs.  If you're familiar with them and have interest in extending this framework to more cosmologies, realizations, or methods, Astropy always welcomes contributions.

## Introduction

The [``astropy.cosmology``](https://docs.astropy.org/en/stable/cosmology/index.html) package provides an object-oriented approach to cosmology that tightly integrates with the rest of the Astropy ecosystem, and units in particular.

With that in mind, the following is the standard set of imports to begin working with the cosmology package.

In [ ]:
from astropy import cosmology
import astropy.units as u

The first thing we will need is to select a cosmology, essentially the physics, in which to work. The most commonly used is a flat $\Lambda$CDM cosmology with a Friedmann–Lemaître–Robertson–Walker (FLRW) metric. We will use it here as well.

Concordant to OOP, in ``astropy.cosmology`` the cosmology types are classes and realizations of those cosmologies are instances of the classes. We can import the flat-$\Lambda$CDM cosmology and examine its public methods and attributes:

In [ ]:
from astropy.cosmology import FlatLambdaCDM  # same as cosmology.FlatLambdaCDM

pubdir = {x for x in dir(FlatLambdaCDM) if not x.startswith("_")}
print("Attributes: ", {x for x in pubdir if not callable(getattr(FlatLambdaCDM, x))})
print("\nMethods: ", {x for x in pubdir if callable(getattr(FlatLambdaCDM, x))})

Many of the attributes are for components to the energy density (e.g. `.Onu0`) or derived quantities like the Hubble time (`.hubble_time`).

The methods are for more involved calculations, like the comoving volume (``.comoving_volume()``).

Before we dive into using the ``FlatLambdaCDM`` cosmology, it is worth noting that there are numerous other cosmologies, many offering alternative dark energy models.

In [ ]:
from astropy.cosmology.core import Cosmology  # the base class

def all_subclasses(cosmo):  # recursively find all subclasses
    yield cosmo.__qualname__
    for c in cosmo.__subclasses__(): yield from all_subclasses(c)

print("Other available cosmologies:", ", ".join(all_subclasses(Cosmology)))

To work with a cosmology we must create an instance, a *realization*, of the cosmology class.
A realization sets single values for the cosmological parameters and allows for numerical computation.

In [ ]:
# make a realization, setting the required H0 & Om0 and overriding the default Tcmb0
cosmo = FlatLambdaCDM(H0=70*u.km/u.s/u.Mpc, Om0=0.3, Tcmb0=2.7*u.K)
cosmo

Now the attributes and methods shown above can be called.

In [ ]:
cosmo.H0

In [ ]:
cosmo.hubble_time

In [ ]:
cosmo.angular_diameter_distance_z1z2(2, 4)

In [ ]:
# these also work on array input
cosmo.comoving_distance([1, 2, 3, 4, 5])

## Built-in Cosmology Realizations


While the cosmology classes support arbitrary realizations, most often we are looking to work with a "standard" cosmology, such as the Planck 2018 best-fit values.

Astropy provides a number of these built-in realizations:

In [ ]:
from astropy.cosmology import parameters

print(parameters.available)

From here on we will work with the ``Planck18`` realization.

In [ ]:
from astropy.cosmology import Planck18; Planck18

The Planck'18 realization sets the cosmology components, the temperature, and also has massive neutrinos.

Astropy ``cosmology`` and ``Quantity``s are built on NumPy, meaning that all methods are vectorized.
We can see this in the following plot showing the Universe's age as a function of redshift.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

z = np.geomspace(0.1, 2e3, num=100)

fig = plt.figure()
ax = fig.add_subplot(xlabel="Redshift", ylabel="Age [Gyr]")
ax.loglog(z, Planck18.age(z))
ax.invert_xaxis()
plt.show();

### Modifying a Realization

Cosmology realizations are immutable, but it's often useful to "perturb" about a cosmology. For this, cosmologies have a ``.clone()`` method that can also override any value, but keeps the remaining values the same. For example,

In [ ]:
newcosmo = Planck18.clone(Om0=0.4, Ob0=0.3, name="perturbed")  # override 
newcosmo

We can see the difference between ``Planck18`` and the new cosmology.

In [ ]:
fig = plt.figure(figsize=(14, 5))
ax = fig.add_subplot(121, title="Cosmology age", xlabel="Redshift", ylabel=r"age  [Gyr]")
ax.semilogx(z, Planck18.age(z))
ax.semilogx(z, newcosmo.age(z))
ax.invert_xaxis()

ax = fig.add_subplot(122, title="Cosmology age perturbation", xlabel="Redshift", ylabel=r"$1 - \frac{\rm{perturbed}}{\rm{Planck'18}}$")
ax.semilogx(z, np.abs(1 - newcosmo.age(z)/Planck18.age(z)))
ax.invert_xaxis()
plt.tight_layout()
plt.show();

## The Default Realization

Consider the ``coordinates.Distance`` function. It is Astropy's flexible method to describe distances and understand the connection between distance, parallax, and other related distance measures.

In [ ]:
from astropy.coordinates import Distance
d = Distance(1 * u.Gpc)

In [ ]:
d.distmod

In [ ]:
d.z  # the redshift

The redshift calculation requires a Cosmology.
When a ``Distance`` is initialized, it accepts the cosmology as a keyword argument, but if one is not provided, it will fall back to a default value. This default cosmology realization is controlled by the [``cosmology.default_cosmology``](https://docs.astropy.org/en/stable/api/astropy.cosmology.default_cosmology.html) global state (for more information see [``ScienceState``](https://docs.astropy.org/en/stable/api/astropy.utils.state.ScienceState.html)).

Lets take a closer look.

In [ ]:
from astropy.cosmology import default_cosmology

In [ ]:
# the public methods
{x for x in dir(default_cosmology) if not x.startswith("_")}

The ``get``/``set`` methods are the standard interface.

``get`` will return the curent default cosmology realization,

In [ ]:
default_cosmology.get()  # to get the default Cosmology

while ``set`` will set the default cosmology realization. If used as a [context-manager](https://docs.python.org/3/reference/compound_stmts.html#the-with-statement), the changed default cosmology realization
will revert to the old default.

In [ ]:
with default_cosmology.set("WMAP5"):
    cosmo = default_cosmology.get()
    # do calculations here
    print("New: ", cosmo.name)

print("Reverted: ", default_cosmology.get().name)

Returning to ``Distance``, we can see how changing the default cosmology realization will impact redshift calculations.

In [ ]:
d = Distance(1 * u.Gpc)
dz = d.z
dz

In [ ]:
with default_cosmology.set("WMAP5"):
    print(f"WMAP5 - Planck18: {100 * (d.z / dz - 1):.3}% difference")

For a stable distance-to-redshift, do all calculations with a set cosmology.

In [ ]:
with default_cosmology.set("Planck13"):
    print(Distance(z=0.1))

## Equivalencies

Equivalencies can be used to convert quantities that are not strictly the same physical type, but in a specific context are interchangable.  

In [ ]:
distance = 105 * (u.Mpc/cosmology.units.littleh)
distance

In [ ]:
# This raises an error because they are different units.
try:
    distance.to(u.Mpc)
except u.UnitConversionError:
    print("Cannot convert incompatible units.")

For cosmologies, Astropy offers a $h$ - $H_0$ equivalency for switching between the unitless and unit-ful Hubble constant. 

In [ ]:
# This succeeds using equivalencies.
H0_70 = 70 * u.km / u.s / u.Mpc

distance.to(u.Mpc, cosmology.units.with_H0(H0_70))

And vice versa

In [ ]:
(150 * u.Mpc).to(u.Mpc/cosmology.units.littleh, cosmology.units.with_H0(H0_70))

This equivalency works for arbitrary Quantities.

In [ ]:
luminosity = 0.49 * u.Lsun * cosmology.units.littleh**-2
luminosity.to(u.Lsun, cosmology.units.with_H0(H0_70))

Note that the equivalency is cosmology dependent.

In [ ]:
print(distance.to(u.Mpc, cosmology.units.with_H0(H0_70)),
      distance.to(u.Mpc, cosmology.units.with_H0(Planck18.H0)), sep="\n")

To perform a number of computations with the same cosmology, use [astropy.units.add_enabled_equivalencies](https://docs.astropy.org/en/stable/api/astropy.units.add_enabled_equivalencies.html?highlight=add_enabled_equivalencies). When done as a ``with`` statement, this equivalency will only be enabled within the block and prevent mistakenly using the Planck18 equivalency when working with, e.g., WMAP data.

In [ ]:
with u.add_enabled_equivalencies(cosmology.units.with_H0(Planck18.H0)):
    print(f"distance: {distance.to(u.Mpc)}",
          f"luminosity: {luminosity.to(u.Lsun)}", sep="\n")

There's a lot of flexibility with equivalencies, including a variety of other built-in equivalencies, useful in difference contexts.  So if you want to know more, you might want to check out the [equivalencies narrative documentation](https://docs.astropy.org/en/stable/units/equivalencies.html) or the [astropy.units.equivalencies reference docs](https://docs.astropy.org/en/stable/units/index.html#module-astropy.units.equivalencies).

# Putting it all together:  a concise example


Let's say we have a data set with an array of luminosity distances and we are interested in finding the corresponding redshift for an array of cosmologies.

For this exercise we'll need a ``astropy.cosmology`` function to invert redshift-dependent function and find the redshift at a value.

In [ ]:
from astropy.cosmology import z_at_value  # only works on scalar inputs

In [ ]:
z_at_value(Planck18.age, 10*u.Gyr)

Our "dataset" will consist of a nice set of measurements

In [ ]:
distances = Distance(np.linspace(45, 2535, 100) * u.Mpc)

Now we iterate over a number of cosmologies, perturbing the matter (and baryonic) parameters.

In [ ]:
perturb_Om0 = np.linspace(-0.2, 0.2, num=50)  # the perturbation
zs = np.empty((50, 100))  # preload derived redshift arrays

for i, cosmo in enumerate(Planck18.clone(Om0=Planck18.Om0+dm, Ob0=Planck18.Om0+dm-0.001) for dm in perturb_Om0):
    # compute the redshift at the luminosity distance
    zs[i] = [z_at_value(cosmo.luminosity_distance, d) for d in distances]

zs.shape

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(title="Perturbed Matter Component", xlabel="Distance [Mpc]", ylabel="z")
ax.plot(distances, zs.T, c="gray", alpha=0.1);
for z, dm in zip(zs[::10, :], perturb_Om0[::10]):
    ax.plot(distances, z, label=r"$\Delta\Omega_{m0}=$" + f"{Planck18.Om0+dm:.3}")
ax.legend()
plt.show();

# Exercises

## Exercise 1

Compute the redshift of matter-radiation equality for the Planck 2018 flat-$\Lambda$CDM realization.
Convert this into a variety of distance measures: the scale factor, luminosity distance, distance modulus, etc.

*Hints*:

- Check out ``z_at_value``
- Astropy cosmology instances have defined methods for calculating the matter and radiation components as a function of redshift. See [the docs](https://docs.astropy.org/en/stable/api/astropy.cosmology.FLRW.html).

In [ ]:
z_at_value?

In [ ]:
# Answer here (z)

In [ ]:
# Answer here (a, d, m-M)